# Labour Data Generator
.csv file is from previous semester's synthetic parts generator

In [2]:
import pandas as pd

# Assume df_cars_real is your existing DataFrame with car lifecycle data
# For demonstration, here's a minimal sample (in practice, use your full dataset):
df_cars_real = pd.read_csv("/content/all_parts.csv")

# Define a mapping from manufacturer/model to a car category
car_category_mapping = {
    "Ford": {
         "Focus": "Sedan",
         "Fusion": "Sedan",
         "Mustang": "Coupe",
         "Explorer": "SUV",
         "F-150": "Truck",
         "Escape": "SUV",
         "Edge": "SUV",
         "Ranger": "Truck"
    },
    "Toyota": {
         "Camry": "Sedan",
         "Corolla": "Sedan",
         "Prius": "Hatchback",
         "RAV4": "SUV",
         "Highlander": "SUV",
         "Tacoma": "Truck",
         "Tundra": "Truck",
         "Avalon": "Sedan"
    },
    "BMW": {
         "3 Series": "Sedan",
         "5 Series": "Sedan",
         "7 Series": "Sedan",
         "X5": "SUV",
         "X3": "SUV",
         "M3": "Coupe",
         "Z4": "Convertible",
         "X6": "SUV"
    },
    "Mercedes-Benz": {
         "C-Class": "Sedan",
         "E-Class": "Sedan",
         "S-Class": "Sedan",
         "GLC": "SUV",
         "GLE": "SUV",
         "A-Class": "Hatchback",
         "CLA": "Sedan",
         "G-Wagon": "SUV"
    }
}

# Define standard labour hours per car category
labour_hours_mapping = {
    "Sedan": 2.0,
    "SUV": 2.5,
    "Truck": 3.0,
    "Coupe": 1.8,
    "Convertible": 2.2,
    "Hatchback": 2.0
}

# Generate labour data based on the existing car lifecycle dataset
labour_data = []
for _, row in df_cars_real.iterrows():
    manufacturer = row["Manufacturer"]
    model = row["Model"]
    year = row["Year"]
    # Look up the car category; default to 'Sedan' if not found
    car_category = car_category_mapping.get(manufacturer, {}).get(model, "Sedan")
    hours = labour_hours_mapping.get(car_category, 2.0)
    labour_data.append({
         "Year": year,
         "Manufacturer": manufacturer,
         "Model": model,
         "Car Category": car_category,
         "Labour Hours": hours
    })

# Convert the labour data into a DataFrame
labour_df = pd.DataFrame(labour_data)

# Export the labour data to CSV (if needed)
labour_df.to_csv("labour_data.csv", index=False)

# Verify the structure
print(labour_df.head())

   Year Manufacturer   Model Car Category  Labour Hours
0  2000       Toyota  Avalon        Sedan           2.0
1  2000       Toyota  Avalon        Sedan           2.0
2  2000       Toyota  Avalon        Sedan           2.0
3  2000       Toyota  Avalon        Sedan           2.0
4  2000       Toyota  Avalon        Sedan           2.0


In [4]:
import copy
import json

# Base labour mapping for repair tasks per parts category.
# (For brevity, only a few tasks per category are shown here. You can extend this to include all tasks.)
labour_mapping = {
    "Engine Components": [
        {
            "task": "Engine Diagnostics",
            "hours": 1.0,
            "description": (
                "Perform a complete diagnostic using OBD tools, visual inspection, and performance tests."
            ),
            "steps": [
                "Read fault codes using an OBD scanner",
                "Perform visual inspection of engine components",
                "Confirm issues with a test drive"
            ]
        },
        {
            "task": "Spark Plug Replacement",
            "hours": 1.2,
            "description": (
                "Remove worn spark plugs, check gap settings, and install new plugs with proper torque."
            ),
            "steps": [
                "Remove ignition coils",
                "Extract old spark plugs",
                "Install new spark plugs"
            ]
        }
    ],
    "Transmission System": [
        {
            "task": "Transmission Fluid Change",
            "hours": 2.0,
            "description": (
                "Drain old transmission fluid, replace the filter if necessary, and refill with new fluid."
            ),
            "steps": [
                "Lift the vehicle and locate the transmission pan",
                "Drain old fluid and remove filter",
                "Reassemble and refill with new fluid"
            ]
        },
        {
            "task": "Clutch Replacement",
            "hours": 3.0,
            "description": (
                "Remove the transmission, replace the clutch disc and pressure plate, then reassemble."
            ),
            "steps": [
                "Remove transmission",
                "Replace clutch assembly",
                "Reinstall transmission"
            ]
        }
    ],
    "Suspension and Steering System": [
        {
            "task": "Wheel Alignment",
            "hours": 1.0,
            "description": (
                "Adjust camber, caster, and toe settings to ensure proper tire wear and handling."
            ),
            "steps": [
                "Measure alignment settings",
                "Adjust settings using alignment equipment",
                "Test drive for confirmation"
            ]
        }
    ],
    "Braking System": [
        {
            "task": "Front Brake Reline",
            "hours": 1.0,
            "description": (
                "Resurface or replace the front brake rotors for effective braking."
            ),
            "steps": [
                "Remove the wheel and brake caliper",
                "Machine or replace the rotor",
                "Reassemble components"
            ]
        }
    ],
    "Electrical and Lighting System": [
        {
            "task": "Battery Replacement",
            "hours": 0.5,
            "description": (
                "Remove the old battery and install a new one with secure electrical connections."
            ),
            "steps": [
                "Disconnect battery terminals",
                "Remove old battery",
                "Install and reconnect new battery"
            ]
        }
    ],
    # ... (Add additional categories and tasks as needed)
}

# Dynamic vehicle-type labour hour multipliers.
# These multipliers adjust base labour hours based on vehicle type differences.
vehicle_type_modifiers = {
    "SUV": {
        "Engine Components": 4.1,
        "Transmission System": 3.05,
        "Suspension and Steering System": 2.2,
        "Braking System": 2.1,
        "Electrical and Lighting System": 1.0,
        # Add additional category multipliers if necessary
    },
    "Sedan": {
        "Engine Components": 4.0,
        "Transmission System": 3.0,
        "Suspension and Steering System": 2.0,
        "Braking System": 2.1,
        "Electrical and Lighting System": 1.0,
    },
    "Hatchback": {
        "Engine Components": 3.9,
        "Transmission System": 2.95,
        "Suspension and Steering System": 1.9,
        "Braking System": 1.95,
        "Electrical and Lighting System": 0.8,
    }
}

def get_dynamic_labour_mapping(vehicle_type, base_mapping):
    """
    Given a vehicle type (SUV, Sedan, Hatchback) and the base labour mapping,
    return a new mapping with labour hours adjusted by the vehicle type multipliers.
    """
    modifiers = vehicle_type_modifiers.get(vehicle_type, {})
    dynamic_mapping = {}

    # Process each category and its tasks
    for category, tasks in base_mapping.items():
        # Get the multiplier for this category; default is 1.0 if not defined
        multiplier = modifiers.get(category, 1.0)
        dynamic_tasks = []
        for task in tasks:
            # Create a deep copy of the task to avoid modifying the original mapping
            dynamic_task = copy.deepcopy(task)
            dynamic_task["hours"] = round(dynamic_task["hours"] * multiplier, 2)
            dynamic_tasks.append(dynamic_task)
        dynamic_mapping[category] = dynamic_tasks

    return dynamic_mapping

# Example usage:
dynamic_suv_labour = get_dynamic_labour_mapping("SUV", labour_mapping)
dynamic_sedan_labour = get_dynamic_labour_mapping("Sedan", labour_mapping)
dynamic_hatchback_labour = get_dynamic_labour_mapping("Hatchback", labour_mapping)

# Output the dynamic labour mappings (formatted as JSON for readability)
print("Dynamic Labour Mapping for SUVs:")
print(json.dumps(dynamic_suv_labour, indent=2))

print("\nDynamic Labour Mapping for Sedans:")
print(json.dumps(dynamic_sedan_labour, indent=2))

print("\nDynamic Labour Mapping for Hatchbacks:")
print(json.dumps(dynamic_hatchback_labour, indent=2))


Dynamic Labour Mapping for SUVs:
{
  "Engine Components": [
    {
      "task": "Engine Diagnostics",
      "hours": 4.1,
      "description": "Perform a complete diagnostic using OBD tools, visual inspection, and performance tests.",
      "steps": [
        "Read fault codes using an OBD scanner",
        "Perform visual inspection of engine components",
        "Confirm issues with a test drive"
      ]
    },
    {
      "task": "Spark Plug Replacement",
      "hours": 4.92,
      "description": "Remove worn spark plugs, check gap settings, and install new plugs with proper torque.",
      "steps": [
        "Remove ignition coils",
        "Extract old spark plugs",
        "Install new spark plugs"
      ]
    }
  ],
  "Transmission System": [
    {
      "task": "Transmission Fluid Change",
      "hours": 6.1,
      "description": "Drain old transmission fluid, replace the filter if necessary, and refill with new fluid.",
      "steps": [
        "Lift the vehicle and locate the